In [ ]:
import pandas as pd
import boto3

bucket = 'titanic-sagemaker'
data_key = 'input'

train_data_location = 's3://{}/{}/{}'.format(bucket, data_key, 'train.csv')
test_data_location = 's3://{}/{}/{}'.format(bucket, data_key, 'test.csv')

train_data = pd.read_csv(train_data_location)
test_data = pd.read_csv(test_data_location)

print(train_data.columns)
print(test_data.columns)

print(train_data.head())
print(test_data.head())

# Variable Notes

# PassengerId: Unique identifier for the passenger
# Survived: 1 if passenger survived 0 otherwise
# Pclass: 1 = Upper, 2 = Middle,3 = Lower
# Name: Passenger's name
# Sex: Gender of the passenger denoted by 'male', 'female'
# Age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
# SibSp: No of siblings / spouses aboard the Titanic
# Parch: No of parents / children aboard the Titanic
# Ticket: Ticket number
# Fare: Passenger fare
# Cabin: Passenger cabin
# Embarked: Port of Embarkation; C = Cherbourg, Q = Queenstown, S = Southampton